# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [24]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [23]:
filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

['/Users/ghost/src/github/jupyterlab/udacity/dataModel/cassandra_modeling/event_data/2018-11-01-events.csv', '/Users/ghost/src/github/jupyterlab/udacity/dataModel/cassandra_modeling/event_data/2018-11-03-events.csv', '/Users/ghost/src/github/jupyterlab/udacity/dataModel/cassandra_modeling/event_data/2018-11-02-events.csv']
['/Users/ghost/src/github/jupyterlab/udacity/dataModel/cassandra_modeling/event_data/.ipynb_checkpoints/2018-11-01-events-checkpoint.csv']


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
full_data_rows_list = [] 
    
for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

15
[['', 'Logged In', 'Walter', 'M', '0', 'Frye', '', 'free', 'San Francisco-Oakland-Hayward, CA', 'GET', 'Home', '1.54092E+12', '38', '', '200', '1.54111E+12', '39'], ['', 'Logged In', 'Kaylee', 'F', '0', 'Summers', '', 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'GET', 'Home', '1.54034E+12', '139', '', '200', '1.54111E+12', '8'], ["Des'ree", 'Logged In', 'Kaylee', 'F', '1', 'Summers', '246.30812', 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'PUT', 'NextSong', '1.54034E+12', '139', 'You Gotta Be', '200', '1.54111E+12', '8'], ['', 'Logged In', 'Kaylee', 'F', '2', 'Summers', '', 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'GET', 'Upgrade', '1.54034E+12', '139', '', '200', '1.54111E+12', '8'], ['Mr Oizo', 'Logged In', 'Kaylee', 'F', '3', 'Summers', '144.03873', 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'PUT', 'NextSong', '1.54034E+12', '139', 'Flat 55', '200', '1.54111E+12', '8'], ['Tamba Trio', 'Logged In', 'Kaylee', 'F', '4', 'Summers', '177.18812', 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'PUT', 'NextSon

In [5]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

12


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
from cassandra.cluster import Cluster

try:
    cluster = Cluster(['127.0.0.1']) #Local
    #cluster = Cluster()
    session = cluster.connect()
    print('connected success')
except Exception as e:
    print(e)

connected success


#### Create Keyspace

In [7]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS public
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor': 1 }"""
                   )
    print('keyspace created')
except Exception as e:
    print(e)

keyspace created


#### Set Keyspace

In [8]:
try:
    session.set_keyspace('public')
    print('keyspace set')
except Exception as e:
    print(e)

keyspace set


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [9]:
query = "drop table artist_by_item"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession = 4

In [10]:
query = "CREATE TABLE IF NOT EXISTS artist_by_item "
query = query + "(artist text, song text, length decimal, sessionId int, itemInSession int, PRIMARY KEY ((sessionId, itemInSession)) );"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_by_item (artist, song, length, sessionId, itemInSession) "
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])) )

#### Do a SELECT to verify that the data have been inserted into each table

In [12]:
query = "Select artist, song, length from artist_by_item where sessionId = 338 and itemInSession = 4; "

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song, row.length)

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)  for userid = 10, sessionid = 182

In [13]:
query = "CREATE TABLE IF NOT EXISTS artist_by_userid_sessionid "
query = query + "(artist text, song text, user text, userId int, sessionId int, PRIMARY KEY ((userid, sessionid), song) );"

try:
    session.execute(query)
except Exception as e:
    print(e)                   

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_by_userid_sessionid (artist, song, user, userId, sessionId) "
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], line[1] + " " + line[4], int(line[10]), int(line[8])) )

In [15]:
query = "Select artist, song, user from artist_by_userid_sessionid where userid = 10 and sessionid = 182; "

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song, row.user)

## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'             

In [16]:
query = "drop table username_by_song"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [17]:
query = "CREATE TABLE IF NOT EXISTS username_by_song "
query = query + "(username text, song text, PRIMARY KEY ((song)) );"

try:
    session.execute(query)
except Exception as e:
    print(e)    

In [18]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO username_by_song (username, song) "
        query = query + " VALUES (%s, %s)"
        session.execute(query, (line[1] + " " + line[4], line[9] ) )

In [26]:
query = "Select username from username_by_song where song = 'All Hands Against His Own'; "

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.username)

Error from server: code=2200 [Invalid query] message="unconfigured table username_by_song"


### Drop the tables before closing out the sessions

In [22]:
try:
    session.execute("drop table IF EXISTS artist_by_item; ")
    session.execute("drop table IF EXISTS artist_by_userid_sessionid; ")
    session.execute("drop table IF EXISTS username_by_song; ")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()